In [1]:
%pip install -q sentence-transformers faiss-cpu datasets transformers fastapi uvicorn pyngrok torch

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess, time

ollama_process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

time.sleep(5)

!ollama pull qwen2.5:0.5b


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.



In [3]:
from dataclasses import dataclass
import os

@dataclass
class Config:
    DATASET_SIZE = 1000
    CHUNK_SIZE = 400
    CHUNK_OVERLAP = 50
    TOP_K_RETRIEVAL = 3

    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

    OLLAMA_MODEL = "qwen2.5:0.5b"
    OLLAMA_URL = "http://localhost:11434"

    DATA_DIR = "/content/data"
    FAISS_INDEX_PATH = "/content/data/faiss.index"
    CHUNKS_PATH = "/content/data/chunks.json"

config = Config()
os.makedirs(config.DATA_DIR, exist_ok=True)


In [5]:
import faiss, numpy as np
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

class DocumentProcessor:
    def __init__(self):
        print("Loading embedding model...")
        self.embedding_model = SentenceTransformer(config.EMBEDDING_MODEL)
        self.chunks = []
        self.index = None

    def load_and_process(self):
        print("Loading TriviaQA dataset...")
        dataset = load_dataset("trivia_qa", "unfiltered.nocontext", split="train")
        dataset = dataset.select(range(config.DATASET_SIZE))

        print("Chunking documents...")
        for item in tqdm(dataset):
            text = f"Question: {item['question']} Answer: {item['answer']['value']}"
            words = text.split()

            chunk_size = config.CHUNK_SIZE // 4
            overlap = config.CHUNK_OVERLAP // 4

            for i in range(0, len(words), chunk_size - overlap):
                chunk = " ".join(words[i:i + chunk_size])
                if len(chunk) > 30:
                    self.chunks.append({"text": chunk})

        print(f"Total chunks: {len(self.chunks)}")

    def build_faiss_index(self):
        print("Creating embeddings...")
        texts = [c["text"] for c in self.chunks]
        embeddings = self.embedding_model.encode(
            texts,
            convert_to_numpy=True,
            show_progress_bar=True
        ).astype("float32")

        print("Building FAISS index...")
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings)

        print(f"FAISS index size: {self.index.ntotal}")
processor = DocumentProcessor()
processor.load_and_process()
processor.build_faiss_index()


📥 Loading embedding model...
📚 Loading TriviaQA dataset...


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

✂️ Chunking documents...


  0%|          | 0/1000 [00:00<?, ?it/s]

✅ Total chunks: 1000
🔢 Creating embeddings...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

🔍 Building FAISS index...
✅ FAISS index size: 1000


In [6]:
class Retriever:
    def __init__(self, chunks, index):
        self.chunks = chunks
        self.index = index
        self.embedder = SentenceTransformer(config.EMBEDDING_MODEL)

    def retrieve(self, query, k):
        q_emb = self.embedder.encode([query]).astype("float32")
        dists, ids = self.index.search(q_emb, k)

        contexts = [self.chunks[i]["text"] for i in ids[0]]
        scores = [1 / (1 + d) for d in dists[0]]

        return contexts, scores

retriever = Retriever(processor.chunks, processor.index)


In [11]:
class LLMHandler:
    def generate(self, question, contexts):
        prompt = f"""
You are a question-answering system.

Answer the question using ONLY the information in the context below.
If the answer is NOT present in the context, say exactly:
"I don't know based on the provided context."

Context:
{contexts}

Question: {question}

Answer:
"""
        r = requests.post(
            f"{config.OLLAMA_URL}/api/generate",
            json={
                "model": config.OLLAMA_MODEL,
                "prompt": prompt,
                "stream": False
            }
        )

        return r.json()["response"].strip()


In [ ]:
class RAGPipeline:
    def query(self, question, top_k):
        start = time.time()

        contexts, scores = retriever.retrieve(question, top_k)

        latency = int((time.time() - start) * 1000)

        # Similarity threshold
        if len(scores) == 0 or max(scores) < 0.6:
            return {
                "question": question,
                "answer": "I don't know based on the provided context.",
                "contexts": contexts,
                "scores": scores,
                "latency_ms": latency
            }

        answer = llm.generate(question, "\n".join(contexts))

        return {
            "question": question,
            "answer": answer,
            "contexts": contexts,
            "scores": scores,
            "latency_ms": latency
        }


INFO:     154.183.132.69:0 - "POST /query HTTP/1.1" 200 OK
INFO:     154.183.132.69:0 - "POST /query HTTP/1.1" 200 OK
INFO:     154.183.132.69:0 - "POST /query HTTP/1.1" 200 OK
INFO:     154.183.132.69:0 - "POST /query HTTP/1.1" 200 OK
INFO:     154.183.132.69:0 - "POST /query HTTP/1.1" 200 OK


In [9]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

app = FastAPI(title="RAG TriviaQA API")

class QueryRequest(BaseModel):
    question: str
    top_k: int = 3

class QueryResponse(BaseModel):
    question: str
    answer: str
    contexts: List[str]
    scores: List[float]
    latency_ms: int

@app.get("/")
def health():
    return {"status": "ok"}

@app.post("/query", response_model=QueryResponse)
def query_api(req: QueryRequest):
    return rag_pipeline.query(req.question, req.top_k)


In [10]:
from pyngrok import ngrok
import uvicorn, threading

ngrok.set_auth_token("?????")

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()


🚀 Public URL: NgrokTunnel: "https://factious-erlene-postethmoid.ngrok-free.dev" -> "http://localhost:8000"


INFO:     Started server process [27841]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     154.183.132.69:0 - "GET / HTTP/1.1" 200 OK
INFO:     154.183.132.69:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     154.183.132.69:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     154.183.132.69:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     154.183.132.69:0 - "POST /query HTTP/1.1" 200 OK
INFO:     154.183.132.69:0 - "POST /query HTTP/1.1" 200 OK
INFO:     154.183.132.69:0 - "POST /query HTTP/1.1" 200 OK


In [13]:
sample_questions = [
    {"question": "What is the capital of France?", "answer": "Paris"},
    {"question": "What is the capital of Kenya?", "answer": "Nairobi"},
    {"question": "Who wrote Romeo and Juliet?", "answer": "William Shakespeare"},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci"},
    {"question": "What is the largest planet in our solar system?", "answer": "Jupiter"},
    {"question": "Who was the first President of the United States?", "answer": "George Washington"},
    {"question": "What year did World War II end?", "answer": "1945"},
    {"question": "Who invented the telephone?", "answer": "Alexander Graham Bell"},
    {"question": "What is the smallest country in the world?", "answer": "Vatican City"},
    {"question": "What is the chemical symbol for gold?", "answer": "Au"}
]


In [14]:
import pandas as pd

def evaluate_rag(sample_questions, rag_pipeline):
    results = []

    for q in sample_questions:
        res = rag_pipeline.query(q["question"], top_k=3)
        
        # Check if any retrieved context contains the answer
        retrieved_correct = any(q["answer"].lower() in ctx.lower() for ctx in res["contexts"])
        
        # Determine answer correctness
        if res["answer"].lower() == q["answer"].lower():
            correctness = "Correct"
        elif q["answer"].lower() in res["answer"].lower():
            correctness = "Partially Correct"
        else:
            correctness = "Incorrect"
        
        results.append({
            "Question": q["question"],
            "Retrieved Context Correct?": "Yes" if retrieved_correct else "No",
            "Generated Answer": res["answer"],
            "Answer Correctness": correctness,
            "Latency (ms)": res["latency_ms"]
        })

    # Convert to DataFrame
    df = pd.DataFrame(results)
    
    # Summary
    total = len(df)
    retrieved_yes = df["Retrieved Context Correct?"].value_counts().get("Yes", 0)
    correct_ans = df["Answer Correctness"].value_counts().get("Correct", 0)
    partially_correct = df["Answer Correctness"].value_counts().get("Partially Correct", 0)
    avg_latency = df["Latency (ms)"].mean()
    
    print("\nEvaluation Summary")
    print(f"Total Questions: {total}")
    print(f"Retrieved Context Correct: {retrieved_yes}/{total} ({retrieved_yes/total*100:.1f}%)")
    print(f"Answers Correct: {correct_ans}/{total} ({correct_ans/total*100:.1f}%)")
    print(f"Partially Correct Answers: {partially_correct}/{total} ({partially_correct/total*100:.1f}%)")
    print(f"Average Latency: {avg_latency:.0f} ms\n")
    
    return df


In [16]:
evaluation_df = evaluate_rag(sample_questions, rag_pipeline)
print(evaluation_df)


✅ Evaluation Summary
Total Questions: 10
Retrieved Context Correct: 2/10 (20.0%)
Answers Correct: 0/10 (0.0%)
Partially Correct Answers: 6/10 (60.0%)
Average Latency: 10887 ms

                                            Question  \
0                     What is the capital of France?   
1                      What is the capital of Kenya?   
2                        Who wrote Romeo and Juliet?   
3                         Who painted the Mona Lisa?   
4    What is the largest planet in our solar system?   
5  Who was the first President of the United States?   
6                    What year did World War II end?   
7                        Who invented the telephone?   
8         What is the smallest country in the world?   
9              What is the chemical symbol for gold?   

  Retrieved Context Correct?  \
0                         No   
1                        Yes   
2                         No   
3                        Yes   
4                         No   
5            